In [1]:
# !pip install --upgrade pip
# !pip install google-generativeai --upgrade
# !pip install pandas

In [2]:
import google.generativeai as gemini

GOOGLE_API_KEY = "**"

gemini.configure(api_key=GOOGLE_API_KEY)
model = gemini.GenerativeModel('gemini-pro')

In [3]:
file_name = "smollm_360M_cosmo"
# llama3.2_1B_cosmo 75.1
# opt_1.3B_cosmo 77.23
# smallama_101M_cosmo 51.65
# smollm_360M_cosmo 76.08

In [4]:
import pandas as pd

test_result = pd.read_csv('./'+file_name+'.csv', encoding='utf-8', index_col=0)

In [5]:
def format_prompt(sample):
	return f"""Prompt:
Based on the provided instructions, please rate how well the story follows the instructions on a scale of 0 to 100.\
Provide only the score.
 
Instruction:
{sample['prompt']}
 
Generated Story:
{sample['gen_story']}
"""

In [6]:
from tqdm import tqdm
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import time

score = []
for i, result in tqdm(test_result.iterrows(), total=len(test_result), leave=False):
    prompt = format_prompt(result)
    response = model.generate_content(prompt,
                                     safety_settings={
                                         HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
                                         HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
                                         HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                                         HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    })
    score.append(int(response.text))
    
    time.sleep(5)

In [7]:
test_result['score'] = score
test_result.to_csv('./'+file_name+'.csv', encoding='utf-8',)

In [8]:
test_result['score'].mean()

np.float64(76.08)